# D21 - Monitoria em Matemática e Probabilidade
## Aula 02 - Conceitos fundamentais

**Professor: Stefano Mozart**

In [ ]:
#@title Execute esta célula antes de iniciar a resolução dos exercícios
#{display-mode: "form"}
import sys

def validate(func, test, input, output):
  res = True
  if input == None:    
    if not equals(test(func()), output):
      res = False
      print(f'Resultado diferente do esperado.\n')
  else:
    output = output if output != None else [True for e in input]
    for i, o in zip(input, output):
      j = func(*i)
      if not equals(test(i, j), o):
        res = False
        print(f'Resultado diferente do esperado para a entrada {i}.\n')
  
  if res:
    print("Parabéns")

def equals(a, b):
  if type(a) == 'pandas.core.frame.DataFrame':    
    return a.eq(b)  
  return a == b

## Criando um banco SQLite

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

O método `create_engine` configura a conexão do SQLAlchemy ao banco de dados utilizando as informações no `dsn` (tipo de banco, endereço IP, porta, usuário, senha, etc). No exemplo abaixo, utilizamos um banco SQLite, que será gravado no arquivo `save_pandas.db`.

O método `connet` realiza a conexão.

In [ ]:
dsn = 'sqlite:///save_pandas.db'
engine = create_engine(dsn, echo=False)
conn = engine.connect()

Criamos um dataframe para testar a inegração do pandas com o SQLAlchemy

In [ ]:
df = pd.DataFrame({
    'id': [1, 2, 3],
    'name' : ['User 1', 'User 2', 'User 3']
})

## Salvando um dataframe no banco

Salvamos o dataframe no banco, numa tabela chamada `user`. Utilizamos a opção `index=False` para não gravar no banco o índice do dataframe e a opção `if_exists='replace'` pra substituir a tabela no banco, caso já exista

In [ ]:
df.to_sql('user', con=conn, index=False, if_exists='replace')

## Carregando um dataframe a partir de dados do banco

Usamos o método `execute` para enviar instruções SQL ao banco de dados

- O método `fetchall` retorna todos os resultados da consulta na forma de tuplas
- O método `read_sql_database`, do pandas, lê uma tabela e cria um dataframe

In [ ]:
conn.execute("Select * from user").fetchall()

[(1, 'User 1'), (2, 'User 2'), (3, 'User 3')]

In [ ]:
user = pd.read_sql_table('user', con=conn)
user

,id,name
0,1,User 1
1,2,User 2
2,3,User 3


Você também pode usar o construtor `panda.DataFrame` para criar um dataframe a partir de uma consulta sql qualquer

In [ ]:
pd.DataFrame(conn.execute("Select name, id from user where id > 1").fetchall(), columns=['Nome', 'CPF'])

,Nome,CPF
0,User 2,2
1,User 3,3


## Executando instruções SQL

O método `execute` também permite instruções do tipo DDL (Data Definition Language), utilizadas para criação ou alteração de objetos

In [ ]:
# Drop, if exsits
conn.execute('DROP TABLE IF EXISTS post')

# Create
create_table_post = """
CREATE TABLE post (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  title TEXT NOT NULL,
  content TEXT,
  user_id INTEGER  
);
"""

conn.execute(create_table_post)

In [ ]:
insert_into_post = """
INSERT INTO post
  (title, content, user_id)
VALUES
  ('Post 1', 'Conteúdo do post 1', 1),
  ('Post 2', 'Conteúdo do post 2', 1),
  ('Post 3', 'Conteúdo do post 3', 3),
  ('Post 4', 'Conteúdo do post 4', 4);
"""

conn.execute(insert_into_post)

# Exercícios

**Exercício 01**

Escreva uma função que realize um INNER JOIN entre as tabelas `user` e `post`. Utilize o método `fetchall` para retornar o resultado.

In [ ]:
def inner_join():
  # Escreva sua resposta aqui
  
  return conn.execute(select_inner_join).fetchall()

In [ ]:
# Utilize este espaço para testar sua solução
inner_join()

[(1, 'User 1', 1, 'Post 1', 'Conteúdo do post 1', 1),
 (1, 'User 1', 2, 'Post 2', 'Conteúdo do post 2', 1),
 (3, 'User 3', 3, 'Post 3', 'Conteúdo do post 3', 3)]

In [ ]:
# Validação
saida = [(1, 'User 1', 1, 'Post 1', 'Conteúdo do post 1', 1),
 (1, 'User 1', 2, 'Post 2', 'Conteúdo do post 2', 1),
 (3, 'User 3', 3, 'Post 3', 'Conteúdo do post 3', 3)]
validate(inner_join, lambda x: x, None, saida)

Parabéns


**Exercício 02:**
Escreva uma função que retorne um dataframe com todos os elementos da tabela `user`, associados aos respectivos elementos da tabela `post` - quando existirem.

- Observação: utilize os seguintes nomes para as colunas: ['UserId', 'UserName', 'PostId', 'PostTitle', 'PostContent']

In [ ]:
def user_and_post():
  # Escreva sua solução aqui aqui
  

In [ ]:
# Utilize este espaço para testar sua solução


In [ ]:
# Validação
saida = r'{"UserId":{"0":1,"1":1,"2":2,"3":3},"UserName":{"0":"User 1","1":"User 1","2":"User 2","3":"User 3"},"PostId":{"0":1.0,"1":2.0,"2":null,"3":3.0},"PostTitle":{"0":"Post 1","1":"Post 2","2":null,"3":"Post 3"},"PostContent":{"0":"Conte\u00fado do post 1","1":"Conte\u00fado do post 2","2":null,"3":"Conte\u00fado do post 3"}}'
validate(user_and_post, lambda x: x.to_json(), None, saida)

Parabéns
